First: the columns X0-X8
------------------------

we try to find if there is a match between those data , lets construct an artificial key value, and look if there is a match between train - test

 - the length of test and train matches perfectly, thats probably a balancing
-linking on those artifical  key we don't match, so we can't use it to transfer the time

**- although its clear that the ID's are not matching...in that sence they simply splitted the database in two parts and have the stats at hand**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt

%matplotlib inline

def grouper (l0,l1,l2,l3,l4,l5,l6,l8):
    gr=l0+'0 '+l1+'1 '+l2+'2 '+l3+'3 '+l4+'4 '+l5+'5 '+l6+'6 '+l8+'8 '
    return gr
    

df_train = pd.read_csv('../input/train.csv')
print('Size of training set: {} rows and {} columns'.format(*df_train.shape))
print('TRAIN',df_train.head(7))
print('record 252',df_train.iloc[252][['ID','X0','X1','X2','X3','X4','X5','X6','X8']])
# sort smallest first
df_train=df_train.sort_values(by='y', axis=0)  
bins= [c for c in df_train.columns if 'X' in c]
print('Number of features TRAIN: {}'.format(len(bins)))

# grouping all types together
df_train['keys']=df_train[['X0','X1','X2','X3','X4','X5','X6','X8']].apply(lambda x: grouper(*x), axis=1)  
df_train_gr=df_train.groupby(by='keys', axis=0).mean()
#print(df_train_gr.sort_values(by='y',axis=0))



#reading test___________________________
df_test = pd.read_csv('../input/test.csv')
print('Size of  TEST: {} rows and {} columns'.format(*df_test.shape))
print('Test',df_test.head(7))
print('record 252 in test',df_test.iloc[251][['ID','X0','X1','X2','X3','X4','X5','X6','X8']])
print('record 252 in test',df_test.ix[df_test['ID']==504][['ID','X0','X1','X2','X3','X4','X5','X6','X8']])
# grouping all types together
df_test['keys']=df_test[['X0','X1','X2','X3','X4','X5','X6','X8']].apply(lambda x: grouper(*x), axis=1)  


#merge two databases____on artificial keys_________________________
result = pd.merge(df_test,df_train, how='left', on='keys', left_on=None, right_on=None,
         left_index=False, right_index=False, sort=True,
         suffixes=['_te', '_tr'], copy=True, indicator=False)
print(result.shape)

# Benching with 1-1 linking > gives NO  >1-1 link
sub = pd.DataFrame()
sub['ID'] = result.ID_te
sub['y'] = result.y
sub2 = sub.groupby(by='ID',axis=0).min()
print('transfer time estimate through link on key', sub2.T)

result = pd.merge(df_test,df_train, how='left', on='ID',suffixes=['_te', '_tr'], copy=True, indicator=False)

print(result[['keys_tr','keys_te','ID']].head())
print(result.shape)

CORRELATION ?
-------------

Yes there is correlation between the data

The correlation of the Y-time versus the data can be very positive, and negative...
the negative correlated parameters like X127 will reduce the time spend on the car-tests...


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

corrtest=bins+['y']
#correlation matrix
corrm=df_train[corrtest].corr()
plt.imshow(corrm,
           interpolation='nearest', cmap=cm.gist_rainbow)
plt.show()
#some rows have no data
#print(np.diag(corrm))
print(len(corrm))
#sort on y
corry=corrm[len(corrm)-1:len(corrm)]
corry=corry.T.sort_values(by='y',axis=0)

ax = corry.plot(kind='bar', title ="Y", figsize=(15, 10), legend=True, fontsize=12)
# so the uncorrelated to y are the time gaining components...
ax = corry[0:30].plot(kind='bar', title ="Y", figsize=(15, 10), legend=True, fontsize=12)

Is there on average a difference between the X-columns in the database TEST versus TRAIN ?
========================================================================

visually there is no big difference
so fe the X127 that is negative correlated will help nearly the most to reduce the time
and the data learned from the training set should be useable for the test set

In [ ]:
from numpy import corrcoef, sum, log, arange
usable_columns = list(set(df_train.columns) - set(['ID', 'y','X0', 'X6', 'X3', 'X4', 'X8', 'X2', 'keys', 'X5', 'X1']))

trainstats=df_train[usable_columns].describe().T

print(trainstats.head())
trainstats['count']=trainstats['count']/4209
trainstatsort=trainstats.sort_values(by='mean',axis=0)

teststats=df_test[usable_columns].describe().T
print(teststats.head())
teststats['count']=teststats['count']/4209
teststatsort=teststats.sort_values(by='mean',axis=0)

stats=pd.merge(trainstats, teststats, how='inner', on=None, left_on=None, right_on=None,
         left_index=True, right_index=True, sort=False,
         suffixes=('_tr', '_te'), copy=True, indicator=False)
statssort=stats.sort_values(by='mean_tr')
#print(stats)

statssort[['mean_tr','mean_te']].tail(50).plot(kind='bar', title ="Y", figsize=(15, 10), legend=True, fontsize=12)

vcovm=np.cov(df_train[usable_columns].T,df_train['y'].T)
df_vcm=pd.DataFrame(vcovm)
diagonalen=pd.DataFrame(np.diag(df_vcm))

statssort[['std_tr','std_te']].tail(50).plot(kind='bar', title ="Y", figsize=(15, 10), legend=True, fontsize=12)

Lets treat the artificial key column as a tfidf txt..
----
we found a big difference in paired values between train and test

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer( ngram_range=(1,2))
tfidftest=tfidf.fit_transform(df_train['keys'].append(df_test['keys']))

print(tfidf)

#tfidf2 = TfidfVectorizer( ngram_range=(1,2))
#tfidf2.fit_transform(df_test['keys'])
#print(tfidf2)

wordnr = pd.DataFrame(pd.Series(list(tfidf.vocabulary_.keys()), index=tfidf.vocabulary_.values()),columns=['woord']) # #print(words)  #woordenboek !
words=np.asarray(wordnr.woord)

#wordnr2 = pd.DataFrame(pd.Series(list(tfidf2.vocabulary_.keys()), index=tfidf2.vocabulary_.values()),columns=['woord']) # #print(words)  #woordenboek !
#words2=np.asarray(wordnr2.woord)

#are there words not existing in the test ?
#print([w for w in words2 if w not in words])
#alot of new unique combinations

df_train[df_train['keys'].str.contains('az0 s1 as2')]
df_test[df_test['keys'].str.contains('az0 s1 as2')]


Find singulars from the keys
----------------------------

In [ ]:
from sklearn.decomposition import TruncatedSVD


print('svd')     
svd = TruncatedSVD(n_components=50, n_iter=30, random_state=42)
tempi=pd.DataFrame(svd.fit_transform(tfidf.transform(df_train['keys'])))
tempi.rename(columns=lambda x: str(x)+'_i', inplace=True) #nog eens zoeken omcolumns te renamen
print('before',df_train.shape)
df_train=df_train.join(tempi,how='inner')
print('tempi',tempi.shape)
print('joined with tempi',df_train.shape)


print('svd2')     
svd2 = TruncatedSVD(n_components=50, n_iter=30, random_state=42)
temp2=pd.DataFrame(svd2.fit_transform(tfidf.transform(df_test['keys'])))
temp2.rename(columns=lambda x: str(x)+'_i', inplace=True) #nog eens zoeken omcolumns te renamen
df_test=df_test.join(temp2,how='inner')
print('test before',df_test.shape)
print('temp2',temp2.shape)
print('joined with temp2',df_test.shape)

X314
---
most positive correlated with y, separates clearly the time
it's one of the most powerfull separators, or when this option is installed it takes alot of time to check...
its an option that is installed in 30% of the cars... thats sounds like fe manual versus automatic, or benzin versus gasoil

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
sns.pairplot(df_train[['X314','X261','X263','0_i','1_i','2_i','3_i','4_i','5_i','X232','X29','X279','y']], hue='X314')

In [ ]:
Option X279 is a huge timegainer, its installed in a minority of carrs

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
sns.pairplot(df_train[['X314','X261','X263','0_i','1_i','2_i','3_i','4_i','5_i','X232','X29','X279','y']], hue='X279')

XGB
---

In [ ]:
y_train = df_train['y'].values

usable_columns = list(set(df_train.columns) - set(['ID', 'y','X0', 'X6', 'X3', 'X4', 'X8', 'X2', 'keys', 'X5', 'X1']))
x_train = df_train[usable_columns]
x_test = df_test[usable_columns]

import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.1, random_state=4242)

d_train = xgb.DMatrix(x_train, label=y_train)
d_valid = xgb.DMatrix(x_valid, label=y_valid)
d_test = xgb.DMatrix(x_test)

params = {}
params['objective'] = 'reg:gamma'
params['eta'] = 0.02
params['max_depth'] = 16

def xgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

clf = xgb.train(params, d_train, 2000, watchlist, early_stopping_rounds=10, feval=xgb_r2_score, maximize=True, verbose_eval=10)

In [ ]:
from sklearn.preprocessing import LabelEncoder

for c in df_train.columns:
    if df_train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(df_train[c].values) + list(df_test[c].values)) 
        df_train[c] = lbl.transform(list(df_train[c].values))
        df_test[c] = lbl.transform(list(df_test[c].values))
        
print(df_train)
df_norm = (df_train - df_train.mean()) / (df_train.max() - df_train.min())
print(df_norm)